# Prompt Exploration: Building the Core Prompt

This notebook iteratively builds a **Single-Shot Prompt** for the Klartext project, following a "Identity -> Rules -> Examples" structure.

**Goal**: Create a robust prompt that reliably simplifies text to **Level I (Very Easy)** standards.

## Plan
1.  **Define Components**: Break the prompt into `Identity`, `Instructions`, and `Examples`.
2.  **Iterate**: Run against fixed test cases in three stages:
    *   *Stage 1*: Identity Only (Baseline)
    *   *Stage 2*: Identity + Instructions (Rules)
    *   *Stage 3*: Identity + Instructions + Few-Shot Examples
3.  **Evaluate**: Compare the outputs to ensure the rules are being followed.

# 1. Setup & Configuration

In [43]:
import os
import time
from dotenv import load_dotenv, find_dotenv
from groq import Groq

# Load .env
try:
    found_path = find_dotenv(usecwd=True)
    if found_path:
        load_dotenv(found_path, override=True)
        print(f"✅ Loaded .env from: {found_path}")
    else:
        print("⚠️ Could not find .env file.")
except Exception as e:
    print(f"⚠️ Error loading .env: {e}")

# Init Groq
groq_api_key = os.getenv("GROQ_API_KEY")
groq_client = None

if groq_api_key:
    groq_client = Groq(api_key=groq_api_key)
    print("✅ Groq Client Initialized")
else:
    print("⚠️ GROQ_API_KEY missing")

MODEL_NAME = "llama-3.1-8b-instant" # Fast model for iteration

✅ Loaded .env from: /Users/alastair/Github/klartext/.env
✅ Groq Client Initialized


In [44]:
# Fixed Test Cases (Reused from Model Exploration)
FIXED_TEST_CASES = {
    "Advice (Safety)": """
Make sure that the area is a safe place, especially if you plan on walking home at night. 
It’s always a good idea to practice the buddy system. Have a friend meet up and walk with you. 
Research the bus, train or streetcar route you plan to take. Check the schedule for both outgoing and return travel. 
Some public transportation ceases to run late at night. Make sure you don't get stuck without a way home.
    """,
    "Procedural (Cooking)": """
Preheat the oven to 375°F (190°C). Grease a 9x13 inch baking pan. 
In a medium bowl, stir together the flour, baking soda, and salt. 
In a large bowl, cream together the butter and sugar until smooth. 
Beat in the eggs one at a time, then stir in the vanilla. 
Gradually blend in the dry ingredients. Stir in the chocolate chips.
    """,
    "Dense (Legal/Formal)": """
The obligations contained herein shall remain in full force and effect indefinitely, 
notwithstanding the termination of this Agreement, until such time as the Confidential Information 
no longer qualifies as confidential under applicable law. 
The Receiving Party agrees to return all physical copies of the Confidential Information upon request.
    """,
    "Light (Popular culture)": """
Shakira Isabel Mebarak Ripoll is a Colombian singer, songwriter, 
dancer, and record producer.
Often referred to as the “Queen of Latin Music,”she has had a major influence 
on Latin American music and helped bring Spanish-language music
to a global audience.
She has won numerous awards, including four Grammy Awards
and fifteen Latin Grammy Awards, with three wins for Song of the Year.
    """
}

In [45]:
def get_completion(system_prompt, user_prompt):
    if not groq_client: return "[No Client]"
    try:
        completion = groq_client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1
        )
        return completion.choices[0].message.content
    except Exception as e:
        return f"[Error: {e}]"

# 2. Prompt Components

We define the three layers of the prompt using the standardized format:
*   `# Identity`
*   `# Instructions`
*   `# Examples` (XML)

In [46]:
PROMPT_IDENTITY = """
# Identity

You are an expert in plain language writing.
You specialise in rewriting text to be accessible 
to people with learning disabilities or low literacy.
"""

PROMPT_INSTRUCTIONS = """
# Core Task 

* Rewrite the input text to be extremely simple and easy to understand.
* Keep the same meaning as the original text.

# Constraints

* Do NOT include any introductory or concluding text (e.g., "Here is the simplified text").
* Output ONLY the simplified text.
* Never output any XML/HTML tags or attributes (no <...>, no id=...).

# Structure & Formatting Rules

* Use clear structure.
* Use bullet points for steps, lists, or multiple items. Otherwise prefer short sentences.
* Add blank lines between every paragraph.
"""

PROMPT_EXAMPLES = """
# Examples
# The following are example pairs.
# Learn the style and constraints from them.
# Do NOT copy the XML tags into your output.

<examples>

  <example id="1">
    <original_text>
Upon arrival at the facility, visitors are required to sign in at the front desk and present valid photo identification.
    </original_text>

    <simplified_text>
When you arrive:

* Go to the front desk.
* Sign in with your name.
* Show your photo ID.
    </simplified_text>
  </example>

  <example id="2">
    <original_text>
The medication should be administered twice daily with food to minimize potential gastrointestinal discomfort.
    </original_text>

    <simplified_text>
Take this medicine two times every day.

Eat food when you take it. This helps your stomach feel better.
    </simplified_text>
  </example>

</examples>
"""

EL_RULES = """
# Plain Language Rules
# Sentence & Length Rules

* Use very short sentences (maximum 10 words per sentence).
* Break up long sentences.
* Keep subjects and verbs close together.

# Vocabulary & Wording Rules

* Use simple, familiar words. Avoid technical, foreign, or formal terms.
* Explain any uncommon or necessary technical words or abbreviations in parentheses the first time they appear.
* Explain complex ideas or uncommon nouns in parentheses.
* Use positive wording. Avoid negations and never use double negatives.
* Replace abstract nouns with concrete, active verbs.

# Tone & Audience Rules

* Prefer active voice. Avoid passive voice whenever possible.
* Address the reader directly using “you”.
* Use a friendly, neutral tone.
* Avoid bureaucratic, legalistic, or commanding language.

# Consistency & Focus Rules

* Remove filler words and unnecessary details. Keep only essential information.
* Use the same words consistently. Do not switch terms for the same thing.
"""


# 3. Stage 1: Prompt with Identity only (Baseline)
Does the model understand the role without specific rules?

In [47]:
# print(f"🧪 Running STAGE 1 (Identity Only) on {MODEL_NAME}...")

# for label, text in FIXED_TEST_CASES.items():
#     # Construct Prompt
#     system_content = PROMPT_IDENTITY
#     user_content = f"Rewrite this text in simple language:\n{text}"
    
#     output = get_completion(system_content, user_content)
    
#     print(f"\n{'='*60}")
#     print(f"📌 {label}")
#     print(f"{'-'*60}")
#     print(output.strip())
#     time.sleep(0.5)

# 4. Stage 2: Prompt with Identity + Instructions
Adding strict rules. Does it follow formatting constraints (bullets, line breaks)?

In [48]:
# print(f"🧪 Running STAGE 2 (Identity + Instructions) on {MODEL_NAME}...")

# for label, text in FIXED_TEST_CASES.items():
#     # Construct Prompt
#     system_content = f"""{PROMPT_IDENTITY}

# {PROMPT_INSTRUCTIONS}"""
    
#     user_content = f"Rewrite this text in simple language:\n{text}"
    
#     output = get_completion(system_content, user_content)
    
#     print(f"\n{'='*60}")
#     print(f"📌 {label}")
#     print(f"{'-'*60}")
#     print(output.strip())
#     time.sleep(0.5)

# 5. Stage 3: Prompt with full context (Identity + Instructions + Examples)
Adding few-shot examples to demonstrate the style. This should be the highest quality.

In [49]:
# print(f"🧪 Running STAGE 3 (Full Context) on {MODEL_NAME}...")

# for label, text in FIXED_TEST_CASES.items():
#     # Construct Prompt
#     system_content = f"""{PROMPT_IDENTITY}

# {PROMPT_INSTRUCTIONS}

# {PROMPT_EXAMPLES}"""
    
#     user_content = f"Rewrite this text in simple language:\n{text}"
    
#     output = get_completion(system_content, user_content)
    
#     print(f"\n{'='*60}")
#     print(f"📌 {label}")
#     print(f"{'-'*60}")
#     print(output.strip())
#     time.sleep(0.5)

# 6. Stage 4: Prompt with full context + Rules from the Klartext-Initiative


In [50]:
#EL_RULES = """
# Plain Language Rules
#
#* Address the reader directly using “you”. Use a friendly, neutral tone.
#* Avoid bureaucratic, legalistic, or commanding language.
#v* Prefer active voice. Avoid passive voice whenever possible.
#* Use positive wording. Avoid negations and never use double negatives.
#* Use simple, familiar words. Avoid technical, foreign, or formal terms.
#* Replace abstract nouns with concrete, active verbs.
#* Explain necessary technical terms or abbreviations the first time they appear.
#* Remove filler words and unnecessary details. Keep only essential information.
#* Use the same words consistently. Do not switch terms for the same thing.
#* Break up long sentences. No sentence longer than 10 words.
#* Keep subjects and verbs close together.
#* Use clear structure. Use bullet points for lists or steps.
#"""

In [51]:
print(f"🧪 Running STAGE 4 (KlarText rules) on {MODEL_NAME}...")

for label, text in FIXED_TEST_CASES.items():
    # Construct Prompt
    system_content = f"""{PROMPT_IDENTITY}

{PROMPT_INSTRUCTIONS}

{EL_RULES}

{PROMPT_EXAMPLES}"""
    
    user_content = f"Rewrite this text in simple language:\n{text}"
    
    output = get_completion(system_content, user_content)
    
    print(f"\n{'='*60}")
    print(f"📌 {label}")
    print(f"{'-'*60}")
    print(output.strip())
    time.sleep(0.5)

🧪 Running STAGE 4 (KlarText rules) on llama-3.1-8b-instant...

📌 Advice (Safety)
------------------------------------------------------------
When you're out, make sure you're in a safe place. 

If you walk home at night, be extra careful. 

Here's how:

* Find a friend to walk with you.
* Meet up with them and walk together.

Also, plan your trip home:

* Check the bus, train, or streetcar route.
* Look at the schedule for going home and coming back.
* Some buses and trains stop running late at night.

📌 Procedural (Cooking)
------------------------------------------------------------
To make this recipe:

1. Turn the oven to 375°F (190°C).
2. Use butter or oil to make the baking pan slippery.

Mix these dry things together in a bowl:

* Flour
* Baking soda
* Salt

Mix these wet things together in another bowl:

* Butter
* Sugar
* Beat the eggs one at a time
* Add vanilla

Now mix the wet and dry things together. 
Add chocolate chips and mix well.

📌 Dense (Legal/Formal)
-------------